In [1]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy

In [2]:
base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01")
dat_path = base.with_suffix(".dat")
task_path = base.with_suffix(".xls")
info_path = base.with_suffix(".yaml")
exists = {"dat_path":dat_path.exists(), "task_path":task_path.exists(), "info_path": info_path.exists()}
if not np.all(list(exists.values())):
    display(exists)
    raise Exception("Missing some input files...")

In [3]:
event_df = pd.read_csv(dat_path, sep="\t", names=['time (ms)', 'family', 'nbre', '_P', '_V', '_L', '_R', '_T', '_W', '_X', '_Y', '_Z'], skiprows=13, dtype=int)
event_df.insert(0, "t", event_df.pop("time (ms)")/1000)
event_df = event_df.sort_values("t").reset_index(drop=True).reset_index(names="poly_evnum")
print(event_df.to_string())

      poly_evnum         t  family  nbre  _P  _V  _L  _R   _T   _W   _X   _Y  _Z
0              0     0.006      10     1   1   1   0   0    1    0    0    0   0
1              1     0.023      10     1   1   1   0   0   36    0    0    0   0
2              2     0.023       6    22   1   1   1   0    2    1    1    1   0
3              3     0.118      10     1   1   1   0   0   38    0    0    0   0
4              4     0.128       1     2   0   0   0   0    0    0    0    0   0
5              5     0.128       1     3   0   0   0   0    0    0    0    0   0
6              6     0.128       1     2   0   0   0   0    0    0    0    0   0
7              7     0.128       1     4   0   0   0   0    0    0    0    0   0
8              8     0.128       1     4   0   0   0   0    0    0    0    0   0
9              9     0.128      10     1   1   1   0   0    3    0    0    0   0
10            10     0.128      10     1   1   1   0   0    2    0    0    0   0
11            11     0.128  

In [4]:
task_df = pd.read_csv(task_path, sep="\t", header=11)
task_df = task_df.rename(columns={task_df.columns[0]: "state_num" })
display(task_df.columns)
task_df

Index(['state_num', 'T1', 'T2', 'T3', 'T4', 'LED1(1,2)', 'LED2(1,3)',
       'LED3(1,4)', 'L1(2,1)', 'L2(2,2)', 'LK1(5,1)', 'LK2(5,2)', 'RD(6,11)',
       'ASND(6,20)', 'PAD(6,22)', 'STR1(13,1)', 'STR2(13,2)', 'TTLP1(15,1)',
       'TTLP2(15,2)', 'TTLP3(15,3)', 'TTLP4(15,4)', 'TTLP5(15,5)',
       'TTLP6(15,6)', 'TTLP7(15,7)', 'TTLP8(15,8)', 'NEXT1', 'NEXT2', 'NEXT3',
       'CT1', 'CT2'],
      dtype='object')

,state_num,T1,T2,T3,T4,"LED1(1,2)","LED2(1,3)","LED3(1,4)","L1(2,1)","L2(2,2)",...,"TTLP4(15,4)","TTLP5(15,5)","TTLP6(15,6)","TTLP7(15,7)","TTLP8(15,8)",NEXT1,NEXT2,NEXT3,CT1,CT2
0,1,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,!ti(36),NaN,NaN,NaN,NaN
1,2,_essais,NaN,NaN,NaN,!on,!on,!on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=1,L1_L=1(28),L2_L=1(26),NaN,NaN
2,3,70,NaN,NaN,NaN,!on,!on,!on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=0(2),!ti,NaN,NaN,NaN
3,4,100,NaN,NaN,NaN,NaN,NaN,on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=0(2),!ti,NaN,NaN,NaN
4,5,400-1400,NaN,NaN,NaN,NaN,NaN,on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=0(2),!ti,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
info = yaml.safe_load(info_path.open("r"))
info

[{'event_name': '{channel_name}',
  'filter': {'channel_name': ['LED(\\d+)', 'ASND', 'TTLP(\\d+)']},
  'state_value': '_V',
  'start_condition': '_V==1',
  'meta': {'count': '_L'}},
 {'event_name': '{channel_name}',
  'filter': {'channel_name': 'L(\\d+)'},
  'start_condition': '_V==1',
  'state_value': '_V'},
 {'event_name': '{channel_name}_lick',
  'filter': {'channel_name': 'LK(\\d+)'},
  'start_condition': '_V==1',
  'state_value': '_V'},
 {'event_name': '{channel_name}_reward',
  'filter': {'channel_name': 'LK(\\d+)'},
  'start_condition': '_P==1',
  'state_value': '_P'},
 {'event_name': 'PAD_V',
  'filter': {'channel_name': 'PAD'},
  'start_condition': '_V==0',
  'state_value': '_V'},
 {'event_name': 'PAD_P',
  'filter': {'channel_name': 'PAD'},
  'start_condition': '_P==0',
  'state_value': '_P'},
 {'event_name': 'poly_linenum_change',
  'filter': {'family': 10},
  'state_value': '_T'},
 {'event_name': 'file_read',
  'filter': {'channel_name': 'RD'},
  'meta': {'read_value': '_T'

In [6]:
# event_df["curr_node"] = event_df["_T"].where(event_df["family"]==10).ffill()
# event_df

In [7]:


channels = pd.Series(task_df.columns).str.extract(r'\s*(?P<channel_name>\w+)\s*\((?P<family>\d+)\s*,\s*(?P<nbre>\d+)\)\s*').dropna(how="all")
channels["family"] = channels["family"].astype(int)
channels["nbre"] = channels["nbre"].astype(int)
channels

    

,channel_name,family,nbre
5,LED1,1,2
6,LED2,1,3
7,LED3,1,4
8,L1,2,1
9,L2,2,2
10,LK1,5,1
11,LK2,5,2
12,RD,6,11
13,ASND,6,20
14,PAD,6,22


In [8]:
event_channels_df = channels.merge(event_df, on=["family", "nbre"], how="right").sort_values("t")
event_channels_df

,channel_name,family,nbre,poly_evnum,t,_P,_V,_L,_R,_T,_W,_X,_Y,_Z
0,NaN,10,1,0,0.006,1,1,0,0,1,0,0,0,0
1,NaN,10,1,1,0.023,1,1,0,0,36,0,0,0,0
2,PAD,6,22,2,0.023,1,1,1,0,2,1,1,1,0
3,NaN,10,1,3,0.118,1,1,0,0,38,0,0,0,0
11,LED2,1,3,11,0.128,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5745,PAD,6,22,5745,1175.145,1,1,1,0,895,843,493,402,2
5746,LK1,5,1,5746,1175.902,1,1,2,1,62,824,0,763,61
5747,LK1,5,1,5747,1175.902,1,1,2,1,62,824,0,763,61
5748,LK1,5,1,5748,1176.052,0,1,2,0,62,825,0,763,62


In [9]:
handle_dict = {}
for item in info:
  if isinstance(item["filter"], list):
    raise Exception("filter list not handled yet")
  event_channels_df["kept"] = True
  for col, val in item["filter"].items():
    if not isinstance(val ,list):
       val = [val]
    event_channels_df["tmp_kept"] = False
    for v in val:
      if col=="__expr":
          event_channels_df["tmp_kept"] = event_channels_df["tmp_kept"] | event_channels_df.eval(v)
      elif col in event_channels_df.columns:
        if col =="channel_name":
          event_channels_df["tmp_kept"] = event_channels_df["tmp_kept"] | event_channels_df["channel_name"].str.fullmatch(v)
        else:
          event_channels_df["tmp_kept"] = event_channels_df["tmp_kept"] | (event_channels_df[col] == v)
    event_channels_df["kept"] = event_channels_df["kept"] & event_channels_df["tmp_kept"]
    event_channels_df = event_channels_df.drop(columns="tmp_kept")
  filtered = event_channels_df.loc[event_channels_df["kept"]].copy()
  event_channels_df = event_channels_df.drop(columns="kept")
  filtered["event_name"] = filtered.apply(lambda row: item["event_name"].format(**row.to_dict()), axis=1)
  new_names = filtered["event_name"].drop_duplicates().to_list()
  for name in new_names:
     if name in handle_dict:
        raise Exception(f'Each item must add different event_names... There is overlap for name {name}')
     handle_dict[name] = dict(info_item=item, ev_df = filtered.loc[filtered["event_name"] == name].drop(columns=["kept"]))
    #  display(filtered.loc[filtered["event_name"] == name])
captured_evnums = set([k for d in handle_dict.values() for k in d["ev_df"]["poly_evnum"]])
unused_evs = set(event_channels_df["poly_evnum"]) - captured_evnums
if len(unused_evs) > 0:
  display(event_channels_df.loc[event_channels_df["poly_evnum"].isin(unused_evs)])
  raise Exception("Some lines are not handled...")
list(handle_dict.keys())

['LED2',
 'LED3',
 'LED1',
 'TTLP1',
 'TTLP3',
 'TTLP2',
 'ASND',
 'TTLP5',
 'TTLP7',
 'TTLP6',
 'TTLP8',
 'TTLP4',
 'L1',
 'LK1_lick',
 'LK1_reward',
 'PAD_V',
 'PAD_P',
 'poly_linenum_change',
 'zone_change',
 'stop']

In [10]:
all=[]
for ev_name, d in handle_dict.items():
    df: pd.DataFrame = d["ev_df"].sort_values("t")
    info_item = d["info_item"]
    if "discarded" in info_item and info_item["discarded"]:
        continue
    if "meta" in info_item:
        r = pd.DataFrame()
        for k, v in info_item["meta"].items():
            r[k] = df.eval(v)
        df["meta"] = r.apply(lambda row: row.to_dict(), axis=1)
    else: 
        df["meta"] = None
    if "state_value" in info_item:
        df["state_value"] = df.eval(info_item["state_value"])
        df["state_grp"] =( df["state_value"] != df["state_value"].shift(1)).cumsum()
        df["next_t"] = df["t"].shift(-1)
        df["next_meta"] = df["meta"].shift(-1)
        def state_grp_agg(d: pd.DataFrame):
            meta = d["next_meta"].iat[0]
            ret = dict(start=d["t"].iat[0], 
                        duration=d["next_t"].iat[-1]-d["t"].iat[0], 
                        meta = meta)
            return pd.Series(d.iloc[0, :].to_dict() | ret)
        df = df.groupby("state_grp").apply(state_grp_agg, include_groups=False)
    else:
        df["start"] = df["t"]
        df["duration"] = np.nan
        df["state_value"] = np.nan
        df["meta"]= None
    if "start_condition" in info_item:
        df=df.loc[df.eval(info_item["start_condition"])]
    all.append(df[["event_name", "start", "duration", "state_value", "meta"]])
    
all = pd.concat(all).sort_values("start")
display(all)
# print(all.loc[all["event_name"] =="LK1_reward"].to_string())
# print(all.loc[all["state_value"] ==12].to_string())

,event_name,start,duration,state_value,meta
1,poly_linenum_change,0.006,0.017,1.0,None
2,poly_linenum_change,0.023,0.095,36.0,None
3,poly_linenum_change,0.118,0.010,38.0,None
4,poly_linenum_change,0.128,0.000,2.0,None
5,poly_linenum_change,0.128,0.070,3.0,None
...,...,...,...,...,...
982,PAD_P,1174.729,0.028,0.0,None
802,PAD_V,1174.922,0.018,0.0,None
984,PAD_P,1175.115,0.030,0.0,None
223,LK1_lick,1175.902,NaN,1.0,None


     event_name     start  duration  state_value  meta
2    LK1_reward    38.692     0.560          1.0  None
4    LK1_reward    39.494     0.058          1.0  None
6    LK1_reward    74.625     0.258          1.0  None
8    LK1_reward    75.427     0.072          1.0  None
10   LK1_reward    83.235     0.484          1.0  None
12   LK1_reward    84.037     0.132          1.0  None
14   LK1_reward   116.788     0.238          1.0  None
16   LK1_reward   117.590     0.038          1.0  None
18   LK1_reward   123.274     0.026          1.0  None
20   LK1_reward   124.076     0.132          1.0  None
22   LK1_reward   132.967     0.449          1.0  None
24   LK1_reward   133.769     0.097          1.0  None
26   LK1_reward   139.575     0.171          1.0  None
28   LK1_reward   140.377     0.000          1.0  None
30   LK1_reward   172.610     0.224          1.0  None
32   LK1_reward   173.412     0.032          1.0  None
34   LK1_reward   228.450     0.516          1.0  None
36   LK1_r

In [11]:

event_items_df = event_channels_df[["channel_name", "family", "nbre"]].drop_duplicates().apply(lambda row: pd.Series(dict(ev1=1, ev2=2)), axis=1)
event_items_df


,ev1,ev2
0,1,2
2,1,2
11,1,2
8,1,2
6,1,2
23,1,2
22,1,2
21,1,2
20,1,2
30,1,2


In [12]:
for i, item in enumerate(info):
  event_items_df[i] = event_channels_df.apply(lambda row: item["event_name"].format(**row.to_dict()), axis=1)

In [13]:
event_triggers = []
# non_channel_event_triggers=[]
for item in info:
    matches = channels
    if "channel_name" in item: 
        regexs = [re.compile(x) for x in (item["channel_name"] if isinstance(item["channel_name"], list) else [item["channel_name"]])]
        matches = matches.loc[matches["channel_name"].]
        for _, row in channels.iterrows():
            row_dict = row.to_dict()
            if np.any([r.fullmatch(row["channel_name"]) for r in regexs]):
                event_triggers.append(item | row_dict | dict( event_name=item["event_name"].format(**row_dict)))
    else:
        event_triggers.append(item | dict(event_name=item["event_name"]))
event_triggers = pd.DataFrame(event_triggers)
event_triggers
            

SyntaxError: invalid syntax (1877809645.py, line 7)

In [ ]:
event_processing = event_triggers.merge(channels, how="outer", on=["channel_name", "family"]) 
if event_processing["event_name"].isna().any():
    display(event_processing[event_processing[["event_name"]].isna()])
    print(f'Warning, channels {event_processing[event_processing["event_name"].isna()]["channel_name"].to_list()} were discarded during processing')
event_processing = event_processing.dropna(subset="event_name")
event_processing

ValueError: You are trying to merge on float64 and object columns for key 'family'. If you wish to proceed you should use pd.concat

In [ ]:
pattern = re.compile(r'\s*(?P<name>\w+)\s*\((?P<family>\d+)\s*,\s*(?P<nbre>\d+)\)\s*')
event_info = []
discarded = []
for col in task_df.columns:
    res = pattern.fullmatch(col)
    if res:
        matches = []
        for cases in info:
            pat = cases["match"]
            col_info = {k:v for k, v in cases.items() if not k=="match"}
            match = re.compile(pat).fullmatch(res["name"])
            if match:
                key_dict = match.groupdict()
                matches.append((col_info, key_dict))
        if len(matches)==0:
            raise Exception(f"No matching cases in info file for column {col}")
        for col_info, key_dict in matches:
            if "event_type" not in col_info and "change_condition" in col_info:
                col_info["event_type"] = "interval"
            if "connected_to" in col_info:
                if isinstance(col_info["connected_to"], list):
                    col_info["connected_to"] = tuple(col_info["connected_to"])
                else:
                    col_info["connected_to"]  = (col_info["connected_to"], )
            if "name" in col_info:
                col_info["name"] = col_info["name"].format(**key_dict)
            else:
                col_info["name"] = res["name"]
            event_info.append(dict(family=int(res["family"]), nbre=int(res["nbre"])) | col_info)
    else:
        discarded.append(col)
event_info = pd.DataFrame(event_info)
print(f"Unmapped channels:  {discarded}")
event_info


KeyError: 'match'

In [ ]:
processed_events_df =event_df.merge(event_info, how="left", on=["family", "nbre"])
processed_events_df["name"] = np.where(processed_events_df["family"]==10, "state_change_"+ processed_events_df["_T"].astype(str), processed_events_df["name"])
processed_events_df["event_type"] = np.where(processed_events_df["family"]==10, "interval", processed_events_df["event_type"])
processed_events_df["change_condition"] = np.where(processed_events_df["family"]==10, '_T==' + processed_events_df["_T"].astype(str), processed_events_df["change_condition"])
if processed_events_df.loc[processed_events_df["family"]==9]["nbre"].nunique()<2:
    processed_events_df = processed_events_df.loc[processed_events_df["family"]!=9]
else:
    raise Exception("Zone events (family=9) are being used but this script does not know how to process them")
processed_events_df["name"] = np.where(processed_events_df["family"]==11, "pause", processed_events_df["name"])
unprocessed_events = processed_events_df.loc[processed_events_df["name"].isna()]
if len(unprocessed_events.index) > 0:
    display(unprocessed_events)
    raise Exception("Problem, some events where not handled")
processed_events_df


,t,family,nbre,_P,_V,_L,_R,_T,_W,_X,_Y,_Z,connected_to,connection_direction,change_condition,event_type,name
0,0.006,10,1,1,1,0,0,1,0,0,0,0,NaN,NaN,_T==1,interval,state_change_1
1,0.023,10,1,1,1,0,0,36,0,0,0,0,NaN,NaN,_T==36,interval,state_change_36
2,0.023,6,22,1,1,1,0,2,1,1,1,0,"(real,)",input,_V==1,interval,PAD
3,0.118,10,1,1,1,0,0,38,0,0,0,0,NaN,NaN,_T==38,interval,state_change_38
4,0.128,1,2,0,0,0,0,0,0,0,0,0,"(real,)",output,_V==1,interval,LED1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6088,1175.902,5,1,1,1,2,1,62,824,0,763,61,"(real,)",output,_L==1,interval,reward1
6089,1175.902,5,1,1,1,2,1,62,824,0,763,61,"(real,)",input,_V==1,interval,lick1
6090,1176.052,5,1,0,1,2,0,62,825,0,763,62,"(real,)",output,_L==1,interval,reward1
6091,1176.052,5,1,0,1,2,0,62,825,0,763,62,"(real,)",input,_V==1,interval,lick1


In [ ]:
processed_events_df["state_count"] = (processed_events_df["family"]==10).cumsum()
processed_events_df["curr_node"] = processed_events_df["_T"].where(processed_events_df["family"]==10).ffill()
processed_events_df

,t,family,nbre,_P,_V,_L,_R,_T,_W,_X,_Y,_Z,connected_to,connection_direction,change_condition,event_type,name,state_count,curr_node
0,0.006,10,1,1,1,0,0,1,0,0,0,0,NaN,NaN,_T==1,interval,state_change_1,1,1.0
1,0.023,10,1,1,1,0,0,36,0,0,0,0,NaN,NaN,_T==36,interval,state_change_36,2,36.0
2,0.023,6,22,1,1,1,0,2,1,1,1,0,"(real,)",input,_V==1,interval,PAD,2,36.0
3,0.118,10,1,1,1,0,0,38,0,0,0,0,NaN,NaN,_T==38,interval,state_change_38,3,38.0
4,0.128,1,2,0,0,0,0,0,0,0,0,0,"(real,)",output,_V==1,interval,LED1,3,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6088,1175.902,5,1,1,1,2,1,62,824,0,763,61,"(real,)",output,_L==1,interval,reward1,1084,45.0
6089,1175.902,5,1,1,1,2,1,62,824,0,763,61,"(real,)",input,_V==1,interval,lick1,1084,45.0
6090,1176.052,5,1,0,1,2,0,62,825,0,763,62,"(real,)",output,_L==1,interval,reward1,1084,45.0
6091,1176.052,5,1,0,1,2,0,62,825,0,763,62,"(real,)",input,_V==1,interval,lick1,1084,45.0


In [ ]:
def ev_num(d:pd.DataFrame):
    ev_name = d.name
    d = d.sort_values("t")
    if (d["event_type"].iat[0] == "timestamp") or pd.isna(d["event_type"].iat[0]):
        return d.assign(ev_num=np.arange(len(d.index)))
    elif  d["event_type"].iat[0] == "interval":
        eval_expr = f'_change = {d["change_condition"].iat[0]}'
        d = d.eval(eval_expr)
        d["ev_num"] = (d["_change"]).cumsum()
        d = d.drop_duplicates(["ev_num", "_change"])
        d=d.drop(columns="_change")
        # display(d)
        return d
    else: raise Exception(f'Unknwon event_type {d["event_type"].iat[0]}')
    
grouped_event_df = processed_events_df.groupby("name").apply(ev_num, include_groups=False).reset_index("name").reset_index(drop=True).sort_values("t")
print(grouped_event_df.to_string())
grouped_event_df

                 name         t  family  nbre  _P  _V  _L  _R   _T   _W   _X   _Y  _Z   connected_to connection_direction change_condition event_type  state_count  curr_node  ev_num
3711   state_change_1     0.006      10     1   1   1   0   0    1    0    0    0   0            NaN                  NaN            _T==1   interval            1        1.0       1
868               PAD     0.023       6    22   1   1   1   0    2    1    1    1   0        (real,)                input            _V==1   interval            2       36.0       1
4242  state_change_36     0.023      10     1   1   1   0   0   36    0    0    0   0            NaN                  NaN           _T==36   interval            2       36.0       1
4345  state_change_38     0.118      10     1   1   1   0   0   38    0    0    0   0            NaN                  NaN           _T==38   interval            3       38.0       1
383              LED2     0.128       1     3   0   0   0   0    0    0    0    0   0     

,name,t,family,nbre,_P,_V,_L,_R,_T,_W,_X,_Y,_Z,connected_to,connection_direction,change_condition,event_type,state_count,curr_node,ev_num
3711,state_change_1,0.006,10,1,1,1,0,0,1,0,0,0,0,NaN,NaN,_T==1,interval,1,1.0,1
868,PAD,0.023,6,22,1,1,1,0,2,1,1,1,0,"(real,)",input,_V==1,interval,2,36.0,1
4242,state_change_36,0.023,10,1,1,1,0,0,36,0,0,0,0,NaN,NaN,_T==36,interval,2,36.0,1
4345,state_change_38,0.118,10,1,1,1,0,0,38,0,0,0,0,NaN,NaN,_T==38,interval,3,38.0,1
383,LED2,0.128,1,3,0,0,0,0,0,0,0,0,0,"(real,)",output,_V==1,interval,3,38.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,PAD,1175.145,6,22,1,1,1,0,895,843,493,402,2,"(real,)",input,_V==1,interval,1084,45.0,1373
3599,lick1,1175.902,5,1,1,1,2,1,62,824,0,763,61,"(real,)",input,_V==1,interval,1084,45.0,208
3598,lick1,1175.902,5,1,1,1,2,1,62,824,0,763,61,"(real,)",input,_V==1,interval,1084,45.0,207
3600,lick1,1176.052,5,1,0,1,2,0,62,825,0,763,62,"(real,)",input,_V==1,interval,1084,45.0,209


In [ ]:
def summarize(grp, d: pd.DataFrame):
    dir = grp["connection_direction"]
    ev_type = grp["event_type"]
    d=d.sort_values("t")
    if len(d.index) == 1:
        if ev_type == "interval":
            return None
        else:
            return dict(start=d["t"].iat[0], duration=pd.NA, num_triggers=pd.NA)
    elif len(d.index) == 2:
        return dict(start=d["t"].iat[0], duration=d["t"].iat[1] - d["t"].iat[0], num_triggers=d["_L"].iat[1] if dir=="output" else pd.NA)
    else:
        display(d)
        raise Exception("unhandled")
grp = ["name", "ev_num", "event_type", "connected_to", "connection_direction"]
res_dict=[]
for grp_l, d in grouped_event_df.groupby(grp, dropna=False):
    grp_dict = {k:v for k, v in zip(grp, grp_l) if k!="ev_num"}
    tmp = summarize(grp_dict, d)
    if not tmp is None:
        res_dict.append(grp_dict | tmp)
res_df = pd.DataFrame(res_dict).sort_values("start")
no_duration_interval_events = res_df[(res_df["event_type"]=="interval") & res_df["duration"].isna()]
if len(no_duration_interval_events.index) > 0:
    display(no_duration_interval_events)
    raise Exception("Interval events with no duration...")

print(res_df.to_string())

         name event_type   connected_to connection_direction     start duration num_triggers
227      LED3   interval        (real,)               output     0.298    0.918            0
0        ASND   interval        (real,)               output     1.216     0.05            1
848     TTLP2   interval  (real, fiber)               output     1.216    0.001            1
899     TTLP3   interval  (real, fiber)               output     1.216    0.146            2
105      LED1   interval        (real,)               output     1.216     0.05            1
785     TTLP1   interval  (real, fiber)               output     1.216    0.096            1
978     TTLP5   interval  (real, fiber)               output    12.268      0.0            1
384       PAD   interval        (real,)                input    12.316    0.118         <NA>
385       PAD   interval        (real,)                input    12.459    0.113         <NA>
386       PAD   interval        (real,)                input    12.596